In [1]:
import pyomo.environ as pyo
import time

# Speichern Sie die aktuelle Zeit vor der Ausführung des Codes
start_time = time.time()

# Model creation
model = pyo.ConcreteModel()

# Creating a list of variable names
drivers=["VER","PER","HAM","ALO","LEC","NOR","SAI","RUS","PIA","STR","GAS","OCO","ALB","BOT","HUL","RIC","ZHO","MAG","SAR"]
teams=["RED","MER","FER","MCL","AST","ALP","ALT","ALF","HAA","WIL"]
variable_names = drivers+teams
#2023 avg
driver_score_avg = {
    "VER": 46.1,"PER": 26.3,"HAM": 23.0,"NOR": 19.9,"ALO": 18.3,"SAI": 17.9,"RUS": 17.5,"LEC": 15.2,"PIA": 12.7,"GAS": 10.5,
    "STR": 10.5,"TSU": 7.3,"ZHO": 6.1,"ALB": 5.7,"OCO": 5.5,"RIC": 4.1,"MAG": 4.0,"HUL": 2.7,"BOT": 2.0,"SAR": 0.1
}
team_score_avg = {
    "RED": 84.0,"MER": 48.2,"FER": 45.8,"MCL": 40.9,"AST": 34.4,"ALP": 21.2,"ALT": 14.2,"ALF": 10.1,"HAA": 9.8,"WIL": 7.8
}
#bet at home quota Bahrain
driver_score_quota = {
    "VER": 2.8, "PER": 2.0, "HAM": 1.7, "NOR": 1.7, "ALO": 1.1, "SAI": 2.2, "RUS": 1.7, "LEC": 2.5, "PIA": 1.1, "GAS": 0.5,
    "STR": 0.6, "TSU": 0.6, "ZHO": 0.3, "ALB": 0.6, "OCO": 0.5, "RIC": 0.6, "MAG": 0.3, "HUL": 0.3, "BOT": 0.3, "SAR": 0.3
}
team_score_quota = {
    "RED": 5.6, "MER": 4.3, "FER": 4.0, "MCL": 3.3, "AST": 3.8, "ALP": 4.5, "ALT": 4.9, "ALF": 4.2, "HAA": 1.9, "WIL": 1.2
}
#quota with a adjustment for linearisation
driver_score_adj_quota = {
    "VER": 42.1, "PER": 29.3, "HAM": 24.2, "NOR": 24.2, "ALO": 13.5, "SAI": 31.5, "RUS": 24.2, "LEC": 36.9, "PIA": 14.4, "GAS": 3.2,
    "STR": 4.4, "TSU": 4.4, "ZHO": 0.8, "ALB": 4.4, "OCO": 3.2, "RIC": 4.4, "MAG": 0.0, "HUL": 0.8, "BOT": 0.8, "SAR": 0.0
}
team_score_adj_quota = {
    "RED": 82.7, "MER": 56.0, "FER": 79.3, "MCL": 44.7, "AST": 20.7, "ALP": 7.4, "ALT": 10.2, "ALF": 1.8, "HAA": 0.9, "WIL": 5.1
}
# Own intuition
driver_score_intuition = {
    "VER": 45.0, "PER": 22.0, "HAM": 22.0, "NOR": 22.0, "ALO": 20.0, "SAI": 22.0, "RUS": 18.0, "LEC": 22.0, "PIA": 18.0, "GAS": 7.0,
    "STR": 10.0, "TSU": 8.0, "ZHO": 6.0, "ALB": 7.0, "OCO": 2.0, "RIC": 8.0, "MAG": 3.0, "HUL": 2.0, "BOT": 2.0, "SAR": 1.5
}
team_score_intuition = {
    "RED": 84.0, "FER": 48.2, "MCL": 45.8, "MER": 40.9, "AST": 34.4, "ALP": 21.2, "ALT": 14.2, "WIL": 10.1, "ALF": 9.8, "HAA": 7.8
}
#Team adjusted by Palmer analysis
driver_score_Palmer = {
    "VER": 46.1, "PER": 26.3, "HAM": 19.3, "NOR": 22.4, "ALO": 18.3, "SAI": 19.1, "RUS": 13.8, "LEC": 16.4, "PIA": 15.1, "GAS": 6.9,
    "STR": 10.5, "TSU": 10.8, "ZHO": 6.0, "ALB": 6.9, "OCO": 1.9, "RIC": 7.6, "MAG": 3.0, "HUL": 1.7, "BOT": 1.8, "SAR": 1.3
}
team_score_Palmer = {
    "RED": 84.0, "FER": 48.2, "MCL": 45.8, "MER": 40.9, "AST": 34.4, "ALP": 21.2, "ALT": 14.2, "WIL": 10.1, "ALF": 9.8, "HAA": 7.8
}
#avg without negative scores
driver_score_noneg = {
    "VER": 46.1, "PER": 31.0, "HAM": 24.2, "NOR": 23.4, "ALO": 22.0, "SAI": 20.5, "RUS": 20.8, "LEC": 20.8, "PIA": 16.4, "GAS": 12.4,
    "STR": 14.0, "TSU": 11.0, "ZHO": 8.8, "ALB": 9.7, "OCO": 11.3, "RIC": 6.1, "MAG": 7.1, "HUL": 4.9, "BOT": 5.6, "SAR": 4.5
}
team_score_noneg = {
    "RED": 84.0, "MER": 48.2, "FER": 45.8, "MCL": 40.9, "AST": 34.4, "ALP": 21.2, "ALT": 14.2, "ALF": 10.1, "HAA": 9.8, "WIL": 7.8
}
# avg - standard deviation
driver_score_low = {
    "VER": 35.41, "PER": 8.08, "HAM": 12.50, "NOR": 1.22, "ALO": 1.54, "SAI": 3.82, "RUS": 4.39, "LEC": 0.08, "PIA": -4.17,
    "GAS": -0.84, "STR": -2.62, "TSU": -4.87, "ZHO": -2.49, "ALB": -5.83, "OCO": -10.62, "RIC": -3.57, "MAG": -6.55, "HUL": -4.53,
    "BOT": -7.77, "SAR": -12.22
}
team_score_low = {
    "RED": 65.14, "MER": 34.92, "FER": 26.38, "MCL": 13.14, "AST": 11.29, "ALP": -0.84, "ALT": -1.79, "ALF": -5.11, "HAA": -2.42, "WIL": -7.82
}
# score 2023 Bahrain result
driver_score_BAH = {
    "VER": 35,"PER": 28,"HAM": 19,"NOR": -1,"ALO": 39,"SAI": 19,"RUS": 16,"LEC": -11,"PIA": -16,"GAS": 20,"STR": 17,"TSU": 8,
    "ZHO": 15,"ALB": 13,"OCO": -17,"RIC": 8,"MAG": 8,"HUL": -1,"BOT": 13,"SAR": 11
}
team_score_BAH = {
    "RED": 78,"MER": 45,"FER": 31,"MCL": -16,"AST": 56,"ALP": 8,"ALT": 17,"ALF": 31,"HAA": 12,"WIL": 25
}
#Cost before Bahrain
driver_cost01={
"VER":30,"PER":20.8,"HAM":19.3,"NOR":23,"ALO":15.8,"SAI":18.5,"RUS":18.9,"LEC":19.1,"PIA":19,"GAS":7.8,"STR":10.7,"TSU":8,
    "ZHO":6.6,"ALB":7,"OCO":7.8,"RIC":9,"MAG":6.2,"HUL":6.4,"BOT":6.4,"SAR":5.5}
team_cost01={
"RED":27.9,"MER":20.1,"FER":19.3,"MCL":23.2,"AST":13.6,"ALP":8.4,"ALT":8.5,"ALF":6.6,"HAA":6.3,"WIL":6.3
}

#Deciding what score should be used
driver_score=driver_score_avg
team_score=team_score_avg
driver_cost=driver_cost01
team_cost=team_cost01

# Creating the pyo.Set with the generated variable names
model.A = pyo.Set(initialize=variable_names)

# Drivers to use
model.x = pyo.Var(model.A, domain=pyo.Binary)

# Constraint
model.max_drivers = pyo.Constraint(expr= sum(model.x[i] for i in drivers)==5 )
model.max_teams = pyo.Constraint(expr= sum(model.x[i] for i in teams)==2 )

model.cost_drivers = pyo.Constraint(expr= sum(model.x[i]*driver_cost[i] for i in drivers)+\
                                    sum(model.x[i]*team_cost[i] for i in teams) <=100 )

# Finde den Fahrer mit dem höchsten driver_score unter den ausgewählten Fahrern
selected_max_driver = max(drivers, key=lambda i: driver_score[i] if model.x[i].value == 1 else float('-inf'))

# Ziel(Objektiv)Funktion
model.objective = pyo.Objective(
    expr=(
        sum(model.x[i] * driver_score[i] * (2 if i == selected_max_driver else 1) for i in drivers) +
        sum(model.x[i] * team_score[i] for i in teams)
    ),
    sense=pyo.maximize
)

# Speichern Sie die aktuelle Zeit nach der Ausführung des Codes
end_time = time.time()
# Berechnen Sie die Differenz zwischen Start- und Endzeit, um die Ausführungszeit zu erhalten
execution_time = end_time - start_time
# Ausgabe der Ausführungszeit
print(f"Programmlaufzeit: {execution_time} s")

Programmlaufzeit: 0.006407022476196289 s


In [2]:
import os
# Speichern Sie die aktuelle Zeit vor der Ausführung des Codes
start_time = time.time()

#os.environ['NEOS_EMAIL'] = 'our.email@stud.tu-darmstadt.de'
#solver_manager = pyo.SolverManagerFactory('neos')
solver_manager = pyo.SolverFactory("gurobi",solver_io="python") ;
results = solver_manager.solve(model, tee=True);

# Speichern Sie die aktuelle Zeit nach der Ausführung des Codes
end_time = time.time()
# Berechnen Sie die Differenz zwischen Start- und Endzeit, um die Ausführungszeit zu erhalten
execution_time = end_time - start_time
# Ausgabe der Ausführungszeit
print(f"Programmlaufzeit: {execution_time} s")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6200U CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3 rows, 29 columns and 58 nonzeros
Model fingerprint: 0x78a0aae3
Variable types: 0 continuous, 29 integer (29 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e-01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+02]
Found heuristic solution: objective 123.2000000
Presolve time: 0.03s
Presolved: 3 rows, 29 columns, 55 nonzeros
Variable types: 0 continuous, 29 integer (29 binary)
Found heuristic solution: objective 147.2000000

Root relaxation: objective 2.388108e+02, 6 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [3]:
# results

In [4]:
 #model.x.pprint()

In [5]:
# Speichern Sie die aktuelle Zeit vor der Ausführung des Codes
start_time = time.time()

print("On basis of the given parameters you should pick the following drivers and teams:")
for i in variable_names: 
    if model.x[i].value==1:
        print(i)
        
# Speichern Sie die aktuelle Zeit nach der Ausführung des Codes
end_time = time.time()
# Berechnen Sie die Differenz zwischen Start- und Endzeit, um die Ausführungszeit zu erhalten
execution_time = end_time - start_time
# Ausgabe der Ausführungszeit
print(f"Programmlaufzeit: {execution_time} s")

On basis of the given parameters you should pick the following drivers and teams:
VER
GAS
ALB
ZHO
MAG
RED
AST
Programmlaufzeit: 0.008659601211547852 s
